In [ ]:
# setup
! pip install bertopic

In [ ]:
#packages for making the model and setting up the data
from bertopic import BERTopic
import pandas as pd
import os

#packages for cleaning
import sys  
!{sys.executable} -m pip install contractions
import contractions  
import re  
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt') 

In [ ]:
#load in the data needed 
filepath_script = os.path.join("Data","all-scripts.csv")
filepath_date = os.path.join("Data","all-detailsepisodes.csv")
df_script = pd.read_csv(filepath_script)   # data containing the scripts 
df_date = pd.read_csv(filepath_date) # data containing the date of releace

#merging the two dataframes by episode id
df = pd.merge(df_script, df_date, on='episodeid')

#removing repeat colunm
df = df.drop('doctorid_y', axis=1)

In [ ]:
#getting rid of NAs, this also gets rid of things not tagged as talk, to check run: print(df['type'].unique())
df = df.dropna()  #319847 rows to 245618 

#making a function that removes special characters from the dates
def clean_date(x):
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@©[\\]^_`{|}~“…”’'), ' ', x)   #Removes special characters
  return x

df["first_diffusion"] = df.first_diffusion.apply(clean_date)

In [ ]:
stop_words = stopwords.words('english')

#making a function that cleans the texts and tokenices it with NLTK
def clean_text(x):
  x = str(x)
  x = x.lower()
  x = contractions.fix(x)
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)   #removes induvial leters and numbers
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@©[\\]^_`{|}~“…”’'), ' ', x)   #removes spetial charecteres
  x = re.sub(r'[%s]' % re.escape("'"), ' ', x)  #removes '
  x = re.sub(r'\d+', ' ', x)     #removes Unicode character category [Nd]
  x = re.sub(r'\n+', ' ', x)     #removes new lines
  x = re.sub(r'\s{2,}', ' ', x)    #removes Unicode whitespace characters
  shortword = re.compile(r'\W*\b\w{1,2}\b') #removes words with only two letters
  x = shortword.sub('', x)
  return x

df['clean_text'] = df.text.apply(clean_text)

In [ ]:
# put the data used for the model in list form, this is also needed for making a timeline
script = df.clean_text.to_list() 
date = df.first_diffusion.to_list()

Code for setting up a model:

In [ ]:
topic_model= BERTopic(language = "english")
topics, probs = topic_model.fit_transform(script)

In [ ]:
#reducing the number of topics to make it easier to work with:
topic_model.reduce_topics(script, nr_topics=1000)   

In [ ]:
#getting an overview of the topics generated
topic_model.get_topic_info() 

Different visualisations:

In [ ]:
topic_model.visualize_barchart(title = "Topic Word Score")

In [ ]:
#visualising the topics
#topic_model.visualize_topics()

In [ ]:
#topics_over_time = basemodel1.topics_over_time(script, date, nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=11) 

In [ ]:
#code for finding related topics, here looking fro the word "war" :
topic_model.find_topics("war", top_n=5) 